In [ ]:
import random
import array
import csv
import pandas as pd
import math
pv1= array.array('d', [])  
pv2= array.array('d', []) 
pv3= array.array('d', [])  
pv4= array.array('d', [])  
pv5= array.array('d', [])  
pv6= array.array('d', [])
load_irr= array.array('d', [])
df=pd.read_csv(r'D:\pv_irr.csv',low_memory=False)
solar_15 = df['s15'].tolist()
solar_18 = df['s18'].tolist()
solar_20 = df['s20'].tolist()
f = open("D:/load_irr_tot.txt")
for i in f:
    load_irr.append(float(i[0:6]))
solar_irr=solar_15*5+solar_18*6+solar_20*9
print (len(load_irr))
print (len(solar_irr))
y=20
pv_year_irr=[5,6,9]
pv_irr=[15,18,20]
replacement_year_irr=[]
Default_Efficiency_irr=[]
Modified_Efficiency_irr=[]
Intelligent_Efficiency_irr=[]
Life_Day_irr=[]
HOMER_Battery_Life_Year_irr=[5.95,6.43,4.39,4.51,4.51,4.51 ]                    # Battery Life_irr and No of Batteries from HOMER
HOMER_No_of_Batteries_Year_irr=[15,15,15,15,15,18,18,18,18,18,18,20,20,20,20,20,20,20,20,20]
HOMER_Battery_Total_Capacity_irr_original=[i*2.4 for i in HOMER_No_of_Batteries_Year_irr]
HOMER_No_of_Replacement_irr=3
#HOMER_No_of_Batteries_replacement_year_irr=[15,18,20,20,20,20,20,20,20]
HOMER_No_of_Batteries_replacement_year_irr=[19,11,12,12,12,16,20,20,20]
HOMER_No_of_Batteries_replacement_year_irr=[i for i in HOMER_No_of_Batteries_replacement_year_irr]
HOMER_Battery_Total_Capacity_irr=[i*2.4 for i in HOMER_No_of_Batteries_replacement_year_irr]
HOMER_Battery_Life_Day_irr=[]
Capacity_Day_irr=[]
Base_Efficiency_Year_irr=[]
def Cap_Day(No_of_Replacement,Battery_Life_Day,Battery_Total_Capacity):
    Capacity_Day=[]
    for n in range(No_of_Replacement+1):                                  
        theeta=[]
        for day in range(int(Battery_Life_Day[n])):
            theeta.append(((float(day)/Battery_Life_Day[n])*90)*3.14159/180)
            capacity=(20*math.exp(0.000258*(day))*math.cos(theeta[day]))+80
            capacity=(capacity*Battery_Total_Capacity[n])/100
            if len(Capacity_Day)<=(20*365)-1:
                Capacity_Day.append(capacity)
            else:
                break
    return Capacity_Day
for n in range(HOMER_No_of_Replacement_irr+1):
    HOMER_Battery_Life_Day_irr.append(round(HOMER_Battery_Life_Year_irr[n]*365))
    HOMER_Battery_Total_Capacity_irr.append(round(HOMER_No_of_Batteries_replacement_year_irr[n]*2.4))

    
Capacity_Day_irr = Cap_Day(HOMER_No_of_Replacement_irr,HOMER_Battery_Life_Day_irr,HOMER_Battery_Total_Capacity_irr)
            
for iteration in range(6):
    No_of_batteries_irr=HOMER_No_of_Batteries_replacement_year_irr[0]
    Bat_irr=[No_of_batteries_irr]                                                                                                           
    Battery_Size_irr=[]
    battery_irr=[]               
    Load_Served_irr=[]
    Load_Shortage_irr=[]

    SoC_excess_irr=[]
    SoC_irr=[]
    Efficiency_Month_irr=[]
    Load_Served_Month_irr=[]
    Excess_Month_irr=[]
    Load_Month_irr=[]
    Life_irr=[]
    Load_Year_irr=[]
    Load_Served_Year_irr=[]
    Excess_Year_irr=[]
    Efficiency_Year_irr=[]
    In_irr=[]
    Out_irr=[]
    Loss_Year_irr=[626,733,685,787,749,853,813,916,874,978,1100,1049,1164,1110,1065,1169,1169,1169,1169,1169]
    AT_irr=[]
    CaP_irr=[]
    Total_Capacity_irr=[]
    load_served_irr=0                          
    Efficiency_irr=0
    soc_irr=100 
    count_irr=0
    p_irr=0
    No_of_Replacement_irr=0
    Lifetime_throughput_irr=905*No_of_batteries_irr
    capacity_irr=No_of_batteries_irr*2.4
    battery_current_irr=capacity_irr
    Battery_Size_irr.append(No_of_batteries_irr)
    Total_Capacity_irr.append(No_of_batteries_irr*2.4)

    for year in range(20):  

        for day in range((year*365),(year*365)+365):

            dod=10
            
            if ((Capacity_Day_irr[day]/capacity_irr)*100)>=105 or ((Capacity_Day_irr[day]/capacity_irr)*100)<78:
                capacity_irr=HOMER_No_of_Batteries_replacement_year_irr[No_of_Replacement_irr+1]*2.4
                

            for hour in range((day*24),(day*24)+24):

                if solar_irr[hour]>=load_irr[hour]:                                                                         # Run-Time Servicing
                    energy_in_irr=solar_irr[hour]-load_irr[hour]
                    battery_current_irr=energy_in_irr+battery_current_irr
                    load_serviced_irr=load_irr[hour]
                    Load_Served_irr.append(load_serviced_irr)
                    shortage_irr=0
                    In_irr.append(energy_in_irr)
                    Out_irr.append(0)

                elif solar_irr[hour]<load_irr[hour]:
                    if (solar_irr[hour]+battery_current_irr-load_irr[hour])>=(dod*capacity_irr)/100:
                        energy_out_irr=load_irr[hour]-solar_irr[hour]
                        battery_current_irr=battery_current_irr-energy_out_irr
                        load_serviced_irr=load_irr[hour]
                        Load_Served_irr.append(load_serviced_irr)
                        shortage_irr=0
                        Out_irr.append(energy_out_irr)
                        In_irr.append(0)
                    else:
                        energy_out_irr=battery_current_irr-(dod*capacity_irr)/100
                        battery_current_irr=battery_current_irr-energy_out_irr
                        load_serviced_irr=solar_irr[hour]+energy_out_irr
                        Load_Served_irr.append(load_serviced_irr)
                        shortage_irr=load_irr[hour]-load_serviced_irr
                        Out_irr.append(energy_out_irr)
                        In_irr.append(0)
                soc_irr=(battery_current_irr/capacity_irr) * 100
                excess_KW_irr=0
                if soc_irr>(Capacity_Day_irr[day]/capacity_irr)*100:
                    excess_percentage_irr=soc_irr-((Capacity_Day_irr[day]/capacity_irr)*100)
                    excess_KW_irr=(excess_percentage_irr*capacity_irr)/100
                    battery_current_irr=Capacity_Day_irr[day]
                    soc_irr=(Capacity_Day_irr[day]/capacity_irr)*100
                    In_irr[len(In_irr)-1]=energy_in_irr-excess_KW_irr
                SoC_excess_irr.append(excess_KW_irr)    
                Load_Shortage_irr.append(shortage_irr)
                battery_irr.append(battery_current_irr)
                SoC_irr.append(soc_irr)

        AT_irr.append((sum(In_irr)+sum(Out_irr)-Loss_Year_irr[year])/2)
        Lifetime_throughput_irr=Lifetime_throughput_irr-AT_irr[year]
                                                                  # Battery Replacement
        if Lifetime_throughput_irr<0:
            Lifetime_throughput_irr=Lifetime_throughput_irr+AT_irr[year]
            No_of_Replacement_irr=No_of_Replacement_irr+1
            balance_irr=AT_irr[year]-Lifetime_throughput_irr
            rem_irr=Lifetime_throughput_irr/AT_irr[year]
            Life_irr.append(count_irr+rem_irr)
            No_of_batteries_irr=HOMER_No_of_Batteries_replacement_year_irr[No_of_Replacement_irr]
            Bat_irr.append(No_of_batteries_irr)
            Lifetime_throughput_irr=905*No_of_batteries_irr
            Lifetime_throughput_irr=Lifetime_throughput_irr-balance_irr
            count_irr=0-rem_irr
            p_irr=p_irr+Life_irr[len(Life_irr)-1]
            replacement_year_irr.append(p_irr)
            Total_Capacity_irr.append(No_of_batteries_irr*2.4)
            capacity_irr=No_of_batteries_irr*2.4

        count_irr=count_irr+1
        In_irr=[]
        Out_irr=[]
    
    theeta_irr=[]
    Days_irr=[]
    Capacity_Day_irr=[]
    for n in range(No_of_Replacement_irr):
        Days_irr.append(round(Life_irr[n]*365))
    Days_irr.append(round(Life_irr[len(Life_irr)-1]*365))
    
    if sum(Days_irr)<7300:
        Days_irr[len(Days_irr)-1]+=7300-sum(Days_irr)
        rem_irr=Lifetime_throughput_irr/AT_irr[year]
    else:
        rem_irr=0

    Capacity_Day_irr = Cap_Day(No_of_Replacement_irr,Days_irr,Total_Capacity_irr)
    
    if iteration==5:
        tote_irr=0
        totls_irr=0
        totl_irr=0
        for n in range(20*8760):                    # Per Year Data Calculation
            if n%8759==0 and n>0:
                Excess_Year_irr.append(tote_irr)
                Load_Served_Year_irr.append(totls_irr)
                Load_Year_irr.append(totl_irr)
                tote_irr=0
                totls_irr=0
                totl_irr=0
            tote_irr=tote_irr+SoC_excess_irr[n]
            totls_irr=totls_irr+Load_Served_irr[n]
            totl_irr=totl_irr+load_irr[n]
            
    #print (solar_irr)

    b=sum(solar_irr)+HOMER_No_of_Batteries_replacement_year_irr[0]*2.4                              # Energy Balance and Efficiency_irr Calculation
    c=sum(Load_Served_irr)+sum(SoC_excess_irr)+battery_irr[(20*8760)-1]
    print("energy balance_irr",b-c,iteration)
    Efficiency_irr=100*(sum(Load_Served_irr)/sum(load_irr))
    
    if iteration==5:
        for year in range(20):
            Efficiency_Year_irr.append(100*(Load_Served_Year_irr[year]/Load_Year_irr[year]))
            Base_Efficiency_Year_irr.append(Efficiency_Year_irr[year])
for n in range(len(Life_irr)):
    Life_Day_irr.append(int(Life_irr[n]*365))

for n in range(y):
    Default_Efficiency_irr.append(round(Efficiency_Year_irr[n],2))

print ('\nOverall Efficiency_irr  :',Efficiency_irr,'%')
print ('Yearly Efficiency_irr      :',Default_Efficiency_irr)
print ('No Of Batteries        :',Bat_irr,sum(Bat_irr))
print ('Replacements           :',No_of_Replacement_irr)
print ('Battery Life_irr Year      :',Life_irr,'yrs',sum(Life_irr))
print ('Battery Life_irr Day       :',Life_Day_irr,sum(Life_Day_irr))

df=pd.read_csv(r'D:\solar(1).csv',low_memory=False)
solar_1 = df['s1'].tolist()
solar_2 = df['s2'].tolist()
solar_3 = df['s3'].tolist()
solar_4 = df['s4'].tolist()
solar_5 = df['s5'].tolist()
solar_6 = df['s6'].tolist()
solar_7 = df['s7'].tolist()
solar_8 = df['s8'].tolist()
solar_9 = df['s9'].tolist()
solar_10 = df['s10'].tolist()
solar_11 = df['s11'].tolist()
solar_12 = df['s12'].tolist()
solar_13 = df['s13'].tolist()
solar_14 = df['s14'].tolist()
solar_15 = df['s15'].tolist()
solar_16 = df['s16'].tolist()
solar_17 = df['s17'].tolist()
solar_18 = df['s18'].tolist()
solar_19 = df['s19'].tolist()
solar_20 = df['s20'].tolist()
df=pd.read_csv(r'D:\load_total.csv',low_memory=False)
load_mg = df['load'].tolist()
solar_mg=solar_3*3+solar_5*2+solar_7*2+solar_9*2+solar_11*2+solar_13*2+solar_15*2+solar_17*2+solar_20*3
y=20
pv_year_mg=[3,2,2,2,2,2,2,5]
pv_mg=[29,32,36,38,42,44,48,50]
replacement_year_mg=[]
Default_Efficiency_mg=[]
Modified_Efficiency_mg=[]
CL_Modified_Efficiency_mg=[]
NCL_Modified_Efficiency_mg=[]
Intelligent_Efficiency_mg=[]
Life_Day_mg=[]
HOMER_Battery_Life_Year_mg=[2.37326,2.38481,2.00879,2.3984,1.92361,2.12548,2.11791,2.32078,2.3423,2.3423]                                      # Battery Life_mg and No of Batteries from HOMER
HOMER_No_of_Batteries_Year_mg=[54,49,61,55,69,74,64,68,82,80,76,83,82,97,89,98,102,103,103,103,103,103]
#for j in range (20):
#    HOMER_No_of_Batteries_Year_mg[j]-=10
#HOMER_Battery_Total_Capacity_mg=[156,144,141.6,175.2,170.4,184.8,172.8,187.2,184.8,237.6,225.6,211.2,259.2,247.2,244.8,271.2,290.4,290.4,290.4,290.4]
HOMER_Battery_Total_Capacity_mg=[i*2.4 for i in HOMER_No_of_Batteries_Year_mg]
HOMER_No_of_Replacement_mg=9
#HOMER_No_of_Batteries_Replacement_Year_mg=[103, 69, 64, 82, 76, 82, 89, 102, 103,103,103] 
#HOMER_No_of_Batteries_Replacement_Year_mg=[103, 50, 40, 40, 50, 70, 89, 102, 110,103,103] 
HOMER_No_of_Batteries_Replacement_Year_mg=[110, 35, 40, 40, 45, 55, 80, 110, 120,103,103] 
HOMER_Battery_Life_Day_mg=[]
Base_Efficiency_Year_mg=[]

#for n in range(HOMER_No_of_Replacement_mg+1):
 #   if n==0:
  #      HOMER_No_of_Batteries_Replacement_Year_mg.append(HOMER_No_of_Batteries_Year_mg[((n+1)*3)-1])
   # elif n<10:
    #    HOMER_No_of_Batteries_Replacement_Year_mg.append(HOMER_No_of_Batteries_Year_mg[(n+1)*2])
    #else:
     #   HOMER_No_of_Batteries_Replacement_Year_mg.append(HOMER_No_of_Batteries_Year_mg[19])
#HOMER_No_of_Batteries_Replacement_Year_mg[0]=50
for n in range(HOMER_No_of_Replacement_mg+1):
    HOMER_Battery_Life_Day_mg.append(round(HOMER_Battery_Life_Year_mg[n]*365))
    HOMER_Battery_Total_Capacity_mg.append(round(HOMER_No_of_Batteries_Replacement_Year_mg[n]*2.4))
#print ('battery life day mg',HOMER_Battery_Life_Day_mg)

Capacity_Day_mg = Cap_Day(HOMER_No_of_Replacement_mg,HOMER_Battery_Life_Day_mg,HOMER_Battery_Total_Capacity_mg)
#print (Capacity_Day_mg)
HOMER_No_of_Batteries_Replacement_Year_mg.append(HOMER_No_of_Batteries_Replacement_Year_mg[len(HOMER_No_of_Batteries_Replacement_Year_mg)-1])
for iteration in range(6):
    No_of_batteries_mg=HOMER_No_of_Batteries_Replacement_Year_mg[0]
    #print(No_of_batteries_mg)
    Bat_mg=[No_of_batteries_mg]                                                                                                           
    Battery_Size_mg=[]
    battery_mg=[]               
    load_served_mg=[]
    Load_Shortage_mg=[]
    SoC_excess_mg=[]
    SoC_mg=[]
    Efficiency_Month_mg=[]
    load_served_Month_mg=[]
    Excess_Month_mg=[]
    Load_Month_mg=[]
    Life_mg=[]
    Load_Year_mg=[]
    load_served_Year_mg=[]
    Excess_Year_mg=[]
    Efficiency_Year_mg=[]
    In_mg=[]
    Out_mg=[]
    Loss_Year_mg=[4337,4738,5024,5113,5549,5755,6108,6310,6596,6889,7153,7380,7525,7816,8209,8453,8647,8650,8652,8654]
    AT_mg=[]
    CaP_mg=[]
    Total_Capacity_mg=[]
    load_serviced_mg=0                          
    Efficiency_mg=0
    soc_mg=100 
    l_mg=0
    p_mg=0
    year_count_mg=0
    No_of_Replacement_mg=0
    Lifetime_throughput_mg=905*No_of_batteries_mg
    capacity_mg=No_of_batteries_mg*2.4
    battery_current_mg=capacity_mg
    Battery_Size_mg.append(No_of_batteries_mg)
    Total_Capacity_mg.append(No_of_batteries_mg*2.4)

    for year in range(20):  
        
        dod = 10
        
        
        for day in range((year*365),(year*365)+365):
            
            if ((Capacity_Day_mg[day]/capacity_mg)*100)>=105 or ((Capacity_Day_mg[day]/capacity_mg)*100)<=78 :
                capacity_mg=HOMER_No_of_Batteries_Replacement_Year_mg[No_of_Replacement_mg+1]*2.4

            for hour in range((day*24),(day*24)+24):
                
                if solar_mg[hour]>=load_mg[hour]:    
                    # Run-Time Servicing
                    energy_in_mg=solar_mg[hour]-load_mg[hour]
                    battery_current_mg=energy_in_mg+battery_current_mg
                    load_serviced_mg=load_mg[hour]
                    load_served_mg.append(load_serviced_mg)
                    shortage_mg=0
                    In_mg.append(energy_in_mg)
                    Out_mg.append(0)

                elif solar_mg[hour]<load_mg[hour]:
                    #print ('hereeee')
                    if (solar_mg[hour]+battery_current_mg-load_mg[hour])>=(dod*capacity_mg)/100:
                        energy_out_mg=load_mg[hour]-solar_mg[hour]
                        battery_current_mg=battery_current_mg-energy_out_mg
                        load_serviced_mg=load_mg[hour]
                        load_served_mg.append(load_serviced_mg)
                        shortage_mg=0
                        Out_mg.append(energy_out_mg)
                        In_mg.append(0)
                    else:
                        energy_out_mg=battery_current_mg-(dod*capacity_mg)/100
                        battery_current_mg=battery_current_mg-energy_out_mg
                        load_serviced_mg=solar_mg[hour]+energy_out_mg
                        load_served_mg.append(load_serviced_mg)
                        shortage_mg=load_mg[hour]-load_serviced_mg
                        Out_mg.append(energy_out_mg)
                        In_mg.append(0)
                soc_mg=(battery_current_mg/capacity_mg) * 100
                excess_KW_mg=0
                if soc_mg>(Capacity_Day_mg[day]/capacity_mg)*100:
                    excess_percentage_mg=soc_mg-((Capacity_Day_mg[day]/capacity_mg)*100)
                    excess_KW_mg=(excess_percentage_mg*capacity_mg)/100
                    battery_current_mg=Capacity_Day_mg[day]
                    soc_mg=(Capacity_Day_mg[day]/capacity_mg)*100
                SoC_excess_mg.append(excess_KW_mg)    
                Load_Shortage_mg.append(shortage_mg)
                battery_mg.append(battery_current_mg)
                SoC_mg.append(soc_mg)

        AT_mg.append((sum(In_mg)+sum(Out_mg)-Loss_Year_mg[year])/2)
        Lifetime_throughput_mg=Lifetime_throughput_mg-AT_mg[year]
        #print (Lifetime_throughput_mg,AT_mg)   # Battery Replacement_mg

        if Lifetime_throughput_mg<0:
            Lifetime_throughput_mg=Lifetime_throughput_mg+AT_mg[year]
            No_of_Replacement_mg=No_of_Replacement_mg+1
            balance_mg=AT_mg[year]-Lifetime_throughput_mg
            rem_mg=Lifetime_throughput_mg/AT_mg[year]
            Life_mg.append(year_count_mg+rem_mg)
            No_of_batteries_mg=HOMER_No_of_Batteries_Replacement_Year_mg[No_of_Replacement_mg]
            Bat_mg.append(No_of_batteries_mg)
            Lifetime_throughput_mg=920*No_of_batteries_mg
            Lifetime_throughput_mg=Lifetime_throughput_mg-balance_mg
            year_count_mg=0-rem_mg
            p_mg=p_mg+Life_mg[len(Life_mg)-1]
            replacement_year_mg.append(p_mg)
            capacity_mg=No_of_batteries_mg*2.4
            Total_Capacity_mg.append(capacity_mg)
            
        year_count_mg=year_count_mg+1
        In_mg=[]
        Out_mg=[]
   
    theeta_mg=[]
    Days_mg=[]
    Capacity_Day_mg=[]
    for n in range(No_of_Replacement_mg):
        Days_mg.append(round(Life_mg[n]*365))
    Days_mg.append(round(Life_mg[len(Life_mg)-1]*365))

    if sum(Days_mg)<7300:
        Days_mg[len(Days_mg)-1]+=7300-sum(Days_mg)
        rem_mg=Lifetime_throughput_mg/AT_mg[year]
    else:
        rem_mg=0

    Capacity_Day_mg = Cap_Day(No_of_Replacement_mg,Days_mg,Total_Capacity_mg)

    if iteration==5:
        tote_mg=0
        totls_mg=0
        totl_mg=0
        for n in range(20*8760):                    # Per Year Data Calculation
            if n%8759==0 and n>0:
                Excess_Year_mg.append(tote_mg)
                load_served_Year_mg.append(totls_mg)
                Load_Year_mg.append(totl_mg)
                tote_mg=0
                totls_mg=0
                totl_mg=0
            tote_mg=tote_mg+SoC_excess_mg[n]
            totls_mg=totls_mg+load_served_mg[n]
            totl_mg=totl_mg+load_mg[n]


    b_mg=sum(solar_mg)+61*2.4                              # Energy Balance and Efficiency_mg Calculation
    c_mg=sum(load_served_mg)+sum(SoC_excess_mg)+battery_mg[(20*8760)-1]
    print (b_mg-c_mg)
    Efficiency_mg=100*(sum(load_served_mg)/sum(load_mg))

    if iteration==5:
        for year in range(20):
            Efficiency_Year_mg.append(100*(load_served_Year_mg[year]/Load_Year_mg[year]))
            Base_Efficiency_Year_mg.append(Efficiency_Year_mg[year])
for n in range(len(Life_mg)):
    Life_Day_mg.append(int(Life_mg[n]*365))

for n in range(y):
    Default_Efficiency_mg.append(round(Efficiency_Year_mg[n],2))
print ('\nOverall Efficiency_mg  :',Efficiency_mg,'%')
print ('Yearly Efficiency_mg      :',Default_Efficiency_mg )  
print ('No Of Batteries        :',No_of_batteries_mg)
print ('Replacements           :',No_of_Replacement_mg)
print ('Battery Life_mg Year      :',Life_mg,'yrs',sum(Life_mg))
print ('Battery Life_mg Day       :',Life_Day_mg,sum(Life_Day_mg))

Life_irr.append(20-sum(Life_irr))
Life_Day_irr.append(7300-sum(Life_Day_irr))
replacement_year_irr=[]
Default_Efficiency_irr=[]
Modified_Efficiency_irr=[]
Intelligent_Efficiency_irr=[]
Battery_Life_Year_irr=Life_irr
Battery_Life_Day_irr= Life_Day_irr
Life_Day_irr=[]
Battery_Total_Capacity_irr=[i * 2.4 for i in Bat_irr]
Base_Efficiency_Year_irr=[]
No_of_Batteries_replacement_year_irr=Bat_irr
y=20
replacement_year_mg=[]
Default_Efficiency_mg=[]
Modified_Efficiency_mg=[]
CL_Modified_Efficiency_mg=[]
NCL_Modified_Efficiency_mg=[]
Intelligent_Efficiency_mg=[]
#Life_Day_mg=[]
Battery_Life_Year_mg=Life_mg 
Battery_Life_Year_mg.append(Battery_Life_Year_mg[len(Battery_Life_Year_mg)-1])                                  
Battery_Life_Day_mg = Life_Day_mg
#print (Life_Day_mg)
Battery_Life_Day_mg.append(Battery_Life_Day_mg[len(Battery_Life_Day_mg)-1])
#No_of_Replacement_mg=9
No_of_Batteries_Replacement_Year_mg=Bat_mg
Battery_Total_Capacity_mg=[i * 2.4 for i in Bat_mg]
Base_Efficiency_Year_mg=[]
#print(len(Battery_Life_Day_mg))
Capacity_Day_mg=[]
Capacity_Day_mg = Cap_Day(No_of_Replacement_mg,Battery_Life_Day_mg,Battery_Total_Capacity_mg)

Capacity_Day_irr=[]
Capacity_Day_irr = Cap_Day(No_of_Replacement_irr,Battery_Life_Day_irr,Battery_Total_Capacity_irr)

for iteration in range(6):
    full_irr=0
    not_full_irr=0
    full_mg=0
    not_full_mg=0
    No_of_batteries_irr=No_of_Batteries_replacement_year_irr[0]
    Bat_irr=[No_of_batteries_irr]
    Battery_Size_irr=[]
    battery_irr=[]               
    Load_Served_irr=[]
    Load_Shortage_irr=[]
    SoC_excess_irr=[]
    SoC_irr=[]
    Efficiency_Month_irr=[]
    Load_Served_Month_irr=[]
    Excess_Month_irr=[]
    Load_Month_irr=[]
    Life_irr=[]
    Load_Year_irr=[]
    Load_Served_Year_irr=[]
    Excess_Year_irr=[]
    Efficiency_Year_irr=[]
    In_irr=[]
    Out_irr=[]
    Loss_Year_irr=[626,733,685,787,749,853,813,916,874,978,1100,1049,1164,1110,1065,1169,1169,1169,1169,1169]
    AT_irr=[]
    CaP_irr=[]
    Total_Capacity_irr=[]
    load_served_irr=0                          
    Efficiency_irr=0
    soc_irr=100 
    p_irr=0
    count_irr=0
    No_of_Replacement_irr=0
    Lifetime_throughput_irr=905*No_of_batteries_irr
    capacity_irr=No_of_batteries_irr*2.4
    Battery_Size_irr.append(No_of_batteries_irr)
    battery_current_irr=capacity_irr                                                                                                           
    Total_Capacity_irr.append(No_of_batteries_irr*2.4)

    No_of_batteries_mg=No_of_Batteries_Replacement_Year_mg[0] 
    #print (No_of_batteries_mg)
    Bat_mg=[No_of_batteries_mg]
    Battery_Size_mg=[]
    battery_mg=[]               
    load_served_mg=[]
    Load_Shortage_mg=[]
    SoC_excess_mg=[]
    SoC_mg=[]
    Efficiency_Month_mg=[]
    load_served_Month_mg=[]
    Excess_Month_mg=[]
    Load_Month_mg=[]
    Life_mg=[]
    Load_Year_mg=[]
    load_served_Year_mg=[]
    Excess_Year_mg=[]
    Efficiency_Year_mg=[]
    In_mg=[]
    Out_mg=[]
    Loss_Year_mg=[4337,4738,5024,5113,5549,5755,6108,6310,6596,6889,7153,7380,7525,7816,8209,8453,8647,8650,8652,8654]
    AT_mg=[]
    CaP_mg=[]
    Total_Capacity_mg=[]
    load_serviced_mg=0                          
    Efficiency_mg=0
    soc_mg=100 
    l_mg=0
    p_mg=0
    year_count_mg=0
    Replacement_mg=0
    Lifetime_throughput_mg=905*No_of_batteries_mg
    capacity_mg=No_of_batteries_mg*2.4
    battery_current_mg=capacity_mg
    Battery_Size_mg.append(No_of_batteries_mg)
    Total_Capacity_mg.append(No_of_batteries_mg*2.4)
    
    for year in range(20):  

        for day in range((year*365),(year*365)+365):
            
            dod=10
        
            for hour in range((day*24),(day*24)+24):

                if solar_irr[hour]>=load_irr[hour]:                                                                         # Run-Time Servicing
                    energy_in_irr=solar_irr[hour]-load_irr[hour]
                    battery_current_irr=energy_in_irr+battery_current_irr
                    load_serviced_irr=load_irr[hour]
                    Load_Served_irr.append(load_serviced_irr)
                    shortage_irr=0
                    In_irr.append(energy_in_irr)
                    Out_irr.append(0)

                elif solar_irr[hour]<load_irr[hour]:
                    if (solar_irr[hour]+battery_current_irr-load_irr[hour])>=(dod*capacity_irr)/100:
                        energy_out_irr=load_irr[hour]-solar_irr[hour]
                        battery_current_irr=battery_current_irr-energy_out_irr
                        load_serviced_irr=load_irr[hour]
                        Load_Served_irr.append(load_serviced_irr)
                        shortage_irr=0
                        Out_irr.append(energy_out_irr)
                        In_irr.append(0)
                    else:
                        energy_out_irr=battery_current_irr-(dod*capacity_irr)/100
                        battery_current_irr=battery_current_irr-energy_out_irr
                        load_serviced_irr=solar_irr[hour]+energy_out_irr
                        Load_Served_irr.append(load_serviced_irr)
                        shortage_irr=load_irr[hour]-load_serviced_irr
                        Out_irr.append(energy_out_irr)
                        In_irr.append(0)
                soc_irr=(battery_current_irr/capacity_irr) * 100
                excess_KW_irr=0
                if soc_irr>(Capacity_Day_irr[day]/capacity_irr)*100:
                    excess_percentage_irr=soc_irr-((Capacity_Day_irr[day]/capacity_irr)*100)
                    excess_KW_irr=(excess_percentage_irr*capacity_irr)/100
                    battery_current_irr=Capacity_Day_irr[day]
                    soc_irr=(Capacity_Day_irr[day]/capacity_irr)*100
                    In_irr[len(In_irr)-1]=energy_in_irr-excess_KW_irr
                SoC_excess_irr.append(excess_KW_irr)    
                Load_Shortage_irr.append(shortage_irr)
                battery_irr.append(battery_current_irr)
                SoC_irr.append(soc_irr)
                
                if solar_mg[hour]>=load_mg[hour]:                      # Run-Time Servicing
                    energy_in_mg=solar_mg[hour]-load_mg[hour]
                    battery_current_mg=energy_in_mg+battery_current_mg
                    load_serviced_mg=load_mg[hour]
                    load_served_mg.append(load_serviced_mg)
                    shortage_mg=0
                    In_mg.append(energy_in_mg)
                    Out_mg.append(0)

                elif solar_mg[hour]<load_mg[hour]:
                    if (solar_mg[hour]+battery_current_mg-load_mg[hour])>=(dod*capacity_mg)/100:
                        energy_out_mg=load_mg[hour]-solar_mg[hour]
                        battery_current_mg=battery_current_mg-energy_out_mg
                        load_serviced_mg=load_mg[hour]
                        load_served_mg.append(load_serviced_mg)
                        shortage_mg=0
                        Out_mg.append(energy_out_mg)
                        In_mg.append(0)
                    else:
                        energy_out_mg=battery_current_mg-(dod*capacity_mg)/100
                        battery_current_mg=battery_current_mg-energy_out_mg
                        load_serviced_mg=solar_mg[hour]+energy_out_mg
                        load_served_mg.append(load_serviced_mg)
                        shortage_mg=load_mg[hour]-load_serviced_mg
                        Out_mg.append(energy_out_mg)
                        In_mg.append(0)
                soc_mg=(battery_current_mg/capacity_mg) * 100
                excess_KW_mg=0
                #print(day)
                if soc_mg>(Capacity_Day_mg[day]/capacity_mg)*100:
                    excess_percentage_mg=soc_mg-((Capacity_Day_mg[day]/capacity_mg)*100)
                    excess_KW_mg=(excess_percentage_mg*capacity_mg)/100
                    battery_current_mg=Capacity_Day_mg[day]
                    soc_mg=(Capacity_Day_mg[day]/capacity_mg)*100
                    In_mg[len(In_mg)-1]=energy_in_mg-excess_KW_mg
                SoC_excess_mg.append(excess_KW_mg)    
                Load_Shortage_mg.append(shortage_mg)
                battery_mg.append(battery_current_mg)
                SoC_mg.append(soc_mg)
                if (Load_Served_irr[hour]==load_irr[hour]):
                    full_irr+=1
                else:
                    not_full_irr+=1
                if (load_served_mg[hour]==load_mg[hour]):
                    full_mg+=1
                else:
                    not_full_mg+=1 

        AT_mg.append((sum(In_mg)+sum(Out_mg)-Loss_Year_mg[year])/2)
        Lifetime_throughput_mg=Lifetime_throughput_mg-AT_mg[year]

        if Lifetime_throughput_mg<0:
            Lifetime_throughput_mg=Lifetime_throughput_mg+AT_mg[year]
            Replacement_mg=Replacement_mg+1
            balance_mg=AT_mg[year]-Lifetime_throughput_mg
            rem_mg=Lifetime_throughput_mg/AT_mg[year]
            Life_mg.append(year_count_mg+rem_mg)
            No_of_batteries_mg=No_of_Batteries_Replacement_Year_mg[Replacement_mg]
            Bat_mg.append(No_of_batteries_mg)
            Lifetime_throughput_mg=905*No_of_batteries_mg
            Lifetime_throughput_mg=Lifetime_throughput_mg-balance_mg
            year_count_mg=0-rem_mg
            p_mg=p_mg+Life_mg[len(Life_mg)-1]
            replacement_year_mg.append(p_mg)
            capacity_mg=No_of_batteries_mg*2.4
            Total_Capacity_mg.append(capacity_mg)

        AT_irr.append((sum(In_irr)+sum(Out_irr)-Loss_Year_irr[year])/2)
        Lifetime_throughput_irr=Lifetime_throughput_irr-AT_irr[year]
        if Lifetime_throughput_irr<0:
            Lifetime_throughput_irr=Lifetime_throughput_irr+AT_irr[year]
            No_of_Replacement_irr=No_of_Replacement_irr+1
            balance_irr=AT_irr[year]-Lifetime_throughput_irr
            rem_irr=Lifetime_throughput_irr/AT_irr[year]
            Life_irr.append(count_irr+rem_irr)
            No_of_batteries_irr=No_of_Batteries_replacement_year_irr[No_of_Replacement_irr]
            Bat_irr.append(No_of_batteries_irr)
            Lifetime_throughput_irr=905*No_of_batteries_irr
            Lifetime_throughput_irr=Lifetime_throughput_irr-balance_irr
            count_irr=0-rem_irr
            p_irr=p_irr+Life_irr[len(Life_irr)-1]
            replacement_year_irr.append(p_irr)
            Total_Capacity_irr.append(No_of_batteries_irr*2.4)

        count_irr=count_irr+1
        In_irr=[]
        Out_irr=[]
                  
        year_count_mg=year_count_mg+1
        In_mg=[]
        Out_mg=[]
        
    theeta_irr=[]
    Days_irr=[]
    Capacity_Day_irr=[]
    for n in range(No_of_Replacement_irr):
        Days_irr.append(round(Life_irr[n]*365))
    Days_irr.append(round(Life_irr[len(Life_irr)-1]*365))
    
    if sum(Days_irr)<7300:
        Days_irr[len(Days_irr)-1]+=7300-sum(Days_irr)
        rem_irr=Lifetime_throughput_irr/AT_irr[year]
    else:
        rem_irr=0
    
    theeta_mg=[]
    Days_mg=[]
    Capacity_Day_mg=[]
    for n in range(Replacement_mg):
        Days_mg.append(round(Life_mg[n]*365))
    Days_mg.append(round(Life_mg[len(Life_mg)-1]*365))
    
    if sum(Days_mg)<7300:
        Days_mg[len(Days_mg)-1]+=7300-sum(Days_mg)
        rem_mg=Lifetime_throughput_mg/AT_mg[year]
    else:
        rem_mg=0


    Capacity_Day_mg = Cap_Day(Replacement_mg,Days_mg,Total_Capacity_mg)
    
    Capacity_Day_irr = Cap_Day(No_of_Replacement_irr,Days_irr,Total_Capacity_irr)

    if iteration==5:
        tote_irr=0
        totls_irr=0
        totl_irr=0
        for n in range(20*8760):                    # Per Year Data Calculation
            if n%8759==0 and n>0:
                Excess_Year_irr.append(tote_irr)
                Load_Served_Year_irr.append(totls_irr)
                Load_Year_irr.append(totl_irr)
                tote_irr=0
                totls_irr=0
                totl_irr=0
            tote_irr=tote_irr+SoC_excess_irr[n]
            totls_irr=totls_irr+Load_Served_irr[n]
            totl_irr=totl_irr+load_irr[n]
        tote_mg=0
        totls_mg=0
        totl_mg=0
        for n in range(20*8760):                    # Per Year Data Calculation
            if n%8759==0 and n>0:
                Excess_Year_mg.append(tote_mg)
                load_served_Year_mg.append(totls_mg)
                Load_Year_mg.append(totl_mg)
                tote_mg=0
                totls_mg=0
                totl_mg=0
            tote_mg=tote_mg+SoC_excess_mg[n]
            totls_mg=totls_mg+load_served_mg[n]
            totl_mg=totl_mg+load_mg[n]


    b=sum(solar_irr)+26*2.4                              # Energy Balance and Efficiency_irr Calculation
    c=sum(Load_Served_irr)+sum(SoC_excess_irr)+battery_irr[(20*8760)-1]
    print("energy balance_irr",b-c,iteration)
    Efficiency_irr=100*(sum(Load_Served_irr)/sum(load_irr))
    
    b_mg=sum(solar_mg)+61*2.4                              # Energy Balance and Efficiency_mg Calculation
    c_mg=sum(load_served_mg)+sum(SoC_excess_mg)+battery_mg[(20*8760)-1]
    print ("energy balance_mg",b_mg-c_mg)
    Efficiency_mg=100*(sum(load_served_mg)/sum(load_mg))
    
    #print ('irr ',sum(Load_Shortage_irr))
    #print ('mg ',sum(Load_Shortage_mg))
    Efficiency=0
    Efficiency=100*(sum(load_served_mg)+sum(Load_Served_irr))/(sum(load_irr)+sum(load_mg))
    
    print ('total efficiency',Efficiency)
    
    if iteration==5:
        for year in range(20):
            Efficiency_Year_irr.append(100*(Load_Served_Year_irr[year]/Load_Year_irr[year]))
            Efficiency_Year_mg.append(100*(load_served_Year_mg[year]/Load_Year_mg[year]))
            Base_Efficiency_Year_mg.append(Efficiency_Year_mg[year])

for n in range(len(Life_irr)):
    Life_Day_irr.append(int(Life_irr[n]*365))
for n in range(len(Life_mg)):
    Life_Day_mg.append(int(Life_mg[n]*365))

print ('excess mg',sum(SoC_excess_mg))
print ('excess irr',sum(SoC_excess_irr))
    
for n in range(y):
    Intelligent_Efficiency_irr.append(round(Efficiency_Year_irr[n],2))
    Default_Efficiency_mg.append(round(Efficiency_Year_mg[n],2))
print ('\n Overall Efficiency_irr  :',Efficiency_irr,'%')
print ('Yearly Efficiency_irr      :',Intelligent_Efficiency_irr)                
print ('No Of Batteries        :',Bat_irr,sum(Bat_irr))
print ('Replacements           :',No_of_Replacement_irr)
print ('Battery Life_irr Year      :',Life_irr,'yrs',sum(Life_irr))
print ('Battery Life_irr Day       :',Days_irr,sum(Life_Day_irr))

print ('\nOverall Efficiency_mg  :',Efficiency_mg,'%')
print ('Yearly Efficiency_mg      :',Default_Efficiency_mg )  
print ('No Of Batteries        :',Bat_mg,sum(Bat_mg))
print ('Replacements           :',Replacement_mg)
print ('Battery Life_mg Year      :',Life_mg,'yrs',sum(Life_mg))
print ('Battery Life_mg Day       :',Days_mg,sum(Life_Day_mg))

L=(sum(Life_irr)+Life_irr[len(Life_irr)-1]-20)
C=300*Bat_irr[0]
O=0;R=0
rep=300
S=20*(Bat_irr[len(Bat_irr)-1])*L
Life_irr.append(20-sum(Life_irr))
for n in range(len(Bat_irr)):
    O=O+(20*(Bat_irr[n]*Life_irr[n]))
    if n>0:
        R=R+(Bat_irr[n]*rep)
T1=C+O+R-S
print ('irrigation')
print (C, O, R, S)
print (T1)

L=(sum(Life_mg)+Life_mg[len(Life_mg)-1]-20)
C=300*Bat_mg[0]
O=0;R=0
rep=300
S=20*(Bat_mg[len(Bat_mg)-1])*L
Life_mg.append(20-sum(Life_mg))
for n in range(len(Bat_mg)):
    O=O+(20*(Bat_mg[n]*Life_mg[n]))
    if n>0:
        R=R+(Bat_mg[n]*rep)
T=C+O+R-S
print ('MG')
print (C, O, R, S)
print (T)

cap=1500*pv_irr[0]
op=0
ad=0
ad=1500*(pv_irr[len(pv_irr)-1]-pv_irr[0])
    
for n in range (len(pv_irr)):
    op=op+500*(pv_irr[n]*pv_year_irr[n])

tot1=cap+op+ad

print ('IRR PV')
print (cap,ad,op)
print (tot1)

cap=1500*pv_mg[0]
op=0
ad=0
ad=1500*(pv_mg[len(pv_mg)-1]-pv_mg[0])
    
for n in range (len(pv_mg)):
    op=op+500*(pv_mg[n]*pv_year_mg[n])

tot=cap+op+ad
print ('MG PV')
print (cap,ad,op)
print (tot)
print ('TOTAL')
print (tot1+tot+T+T1)

print (full_irr+not_full_irr)
print (full_mg+not_full_mg)